In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.feature_selection import SelectFromModel

In [2]:
df = pd.read_csv('1_Merged_Data.csv')

In [3]:
#####################
# Check missing values

In [4]:
vars_with_missing = []

for f in df.columns:
    missings = df[f].isnull().sum() + len(df[df[f]=='MISSING'])
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings/df.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))

Variable curnt_insurer has 859 records (34.98%) with missing values
Variable time_w_carr has 68 records (2.77%) with missing values
Variable prior_bi has 68 records (2.77%) with missing values
Variable veh_lien_cnt has 2451 records (99.80%) with missing values
Variable credit_score has 108 records (4.40%) with missing values
In total, there are 5 variables with missing values


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [5]:
df = df.drop(columns= ['veh_lien_cnt', 'curnt_insurer'])

In [6]:
df['credit_score'].fillna(df['credit_score'].mean(),inplace=True)
df['time_w_carr'].fillna(df['time_w_carr'].mode()[0],inplace=True)
df['prior_bi'].fillna(df['prior_bi'].mode()[0],inplace=True)

In [7]:
df['target'] =df['future_clm_ind']
df = df.drop(columns = ['future_clm_ind'])

In [8]:
def basic_details(df):
    b = pd.DataFrame()
    b['N unique value'] = df.nunique()
    b['dtype'] = df.dtypes
    return b

In [9]:
basic_details(df)

,N unique value,dtype
hhld_id,2456,int64
drvr_cnt,4,int64
veh_cnt,4,int64
min_age,2305,float64
hoh_age,2394,float64
max_age,2395,float64
avg_age,2398,float64
min_mon_lic,2304,float64
hoh_mon_lic,2388,float64
max_mon_lic,2389,float64


In [10]:
def category_type(df):
    col = df.columns
    for i in col:
        if df[i].nunique()<=16:
            df[i] = df[i].astype('category')
category_type(df)

In [11]:
tot_cat_col = list(df.select_dtypes(include=['category']).columns)

In [12]:
def OHE(df, column):
    cat_col = column
    #cat_col = df.select_dtypes(include =['category']).columns
    len_df = df.shape[0]
    c2,c3 = [],{}
    
    print('Categorical feature',len(column))
    for c in cat_col:
        if df[c].nunique()>2 :
            c2.append(c)
            c3[c] = 'ohe_'+c
    
    df = pd.get_dummies(df, prefix=c3, columns=c2,drop_first=True)

    print('Train',df.shape)
    return df

In [13]:
train_1 = OHE(df,tot_cat_col)

Categorical feature 39
Train (2456, 137)


In [14]:
train_1.to_csv('2_Cleaned_Data.csv', index = False)